# Event-based simulation on `DynapSE` hardware

## Hardware basics
The layer uses the DynapSE processor, which consists of 4 chips. Each chips has 4 cores of 256 neurons. The chips, as well as each core in a chip and each neuron in a core are identified with an ID between 0 and 3 or 0 and 256, respectively. However, for this layer the neurons are given logical IDs from 0 to 4095 that range over all neurons. In other words the logical neuron ID is $1024 \cdot \text{ChipID} + 256 \cdot \text{CoreID} + \text{NeuronID}$.


## Setup
### Connecting to Cortexcontrol
In order to work interface the DynapSE chip, this layer relies on `cortexcontrol`. It should be accessed via an `RPyC` connection. In order to run some examples from within this jupyter notebook, we will do the latter. For this we start `cortexcontrol` and run the following commands in its console (not in this notebook):

In [ ]:
import rpyc.utils.classic

c = rpyc.utils.classic.SlaveService()
from rpyc.utils.server import OneShotServer

t = OneShotServer(c, port=1300)
print("RPyC: Ready to start.")
t.start()

If the `cortexcontrol` console prints <br>
<i>"RPyC: Ready to start."</i> <br>
and nothing else, it is ready.

### Using DynapseControl

## Import

Import the class with the following command:

In [2]:
# - Import recurrent RecDIAF layer
from rockpool.layers import RecDynapSE

This might take a while until the `dynapse_control` module has prepared the hardware.

## Instantiation

| Argument | Type | Default | Meaning |
|----------|------|---------|---------|
| `weights_in` | 2D-`ndarray`| - | Input weights (required) |
| `weights_rec` | 2D-`ndarray` | - | Recurrent weights (required) |
| `neuron_ids` | 1D-`ArrayLike` | `None` | IDs of the layer neurons |
| `virtual_neuron_ids` | 1D-`ArrayLike` | `None` | IDs of the virtual (input) neurons |
| `dt` | `float` | `2e-5` | Time step |
| `max_num_trials_batch` | `int` | `None` | Maximum number of trials in individual batch |
| `max_batch_dur` | `float` | `None` | Maximum duration time of individual batch |
| `max_num_timesteps` | `int` | `None` | Maximum number of time steps in individual batch |
| `max_num_events_batch` | `int` | `None` | Maximum number of input Events during individual batch |
| `l_input_core_ids` | `ArrayLike` | `[0]` | IDs of cores that receive input spikes |
| `input_chip_id` | `int` | 0 | Chip that receives input spikes |
| `clearcores_list` | `ArrayLike` | `None` | IDs of cores to be reset |
| `controller` | `DynapseControl` | `None` | `DynapseControl` instance |
| `rpyc_port` | `int` or `None` | `None` | Port for RPyC connection |
| `name` | `str` | "unnamed" | Layer name |

`weights_in` and `weights_rec` are the input and recurrent weights and have to be provided as 2D-arrays. `weights_in` determines the layer's dimensions `nSIzeIn` and `size`. `weights_rec` has to be of size `size` x `size`. Each weight must be an integer (positive or negative). Furthermore, the sum over the absolute values of the elements in any given column in `weights_in` plus the sum over the absolute values of elements in the corresponding column of `weights_rec` must be les or equal to 64. $\sum_{i} |$ `weights_in` $_{ik}| + \sum_{j}|$ `weights_rec` $_{jk}| \leq 64~\forall k$. This is due to limitations of the hardware and cannot be circumvented.

You can choose which physical and virtual neurons are used for the layer by passing their IDs in `vnLayerNeuronIDs` and `vnVirtualNeuronIDs`, which are 1D array-like objects with integers between 0 and 1023 or 0 and 4095, respectively. All neurons with IDs from $j \cdot 256$ to $(j+1) \cdot 256, \ j \in {0,1,..15}$ belong to the same core (with core ID $j \ mod \ 4$). All neurons with IDs from $j \cdot 1024$ to $(j+1) \cdot 1024, \ j \in {0,..,4}$ belong to same chip (with chip ID $j$). With this in mind you can allocate neurons to specific chips and cores. 

<!---The n-th entry of `vnLayerNeuronIDs` corresponds to n-th layer neuron, the n-th entry of `vnVirtualNeurons` corresponds to the n-th input channel of the layer. As described below, under <i>Neurons</i>, it is recommended that the IDs of the physical neurons be all different to the IDs of the virtual neurons. If these arguments are `None`, the neurons will be automatically allocated. -->

In order for the to layer to function as expected you should stick to the following two rules:
- **Neurons that receive external input should be on different cores than  neurons that receive input from other layer neurons. As a consequence, each neuron should not receive both types of input.** The cores with neurons that receive external inputs are set with `l_input_core_ids`.
- **All neurons that receive external input should be on the same chip.** This chip is set with `input_chip_id` and is 0 by default.


*These rules are quite restrictive and it is possible to set them less strictly. Contact Felix if needed.*

`.dt` is a positive `float` that on the one hand sets the discrete layer evolution timestep, as in other layers, but on the other hand also corresponds to the smallest (nonzero) interval between input events that are sent to the chip. It needs to be larger than $1.11 \cdot 10^{-9}$ (seconds). Below, under *Choosing dt* you can find some more thoughts on how to choose this value.

Evolution is automatically split into batches, the size of which is determined by `max_num_events_batch`, `max_num_timesteps`, `max_batch_dur` and `max_num_trials_batch`, which control the maximum number of events, number of timesteps, duration or number of trials in a batch.  All of them can be set to `None`, which corresponds to setting no limit, except for `max_num_events_batch`, where the limit will be set to 65535. If both `max_num_timesteps` and `max_batch_dur` are not `None`, the `max_batch_dur` will be ignored. `max_num_trials_batch` will only have an effect when the input time series to the `evolve` method contains a `trial_start_times` attribute. For more details on how evolutions are split into batches see <i>Evolution in batches</i>.

The list `clearcores_list` contains the IDs of the cores where (presynaptic) connections should be reset on instantiation. Ideally this contains all the cores that are going to be used for this `RecDynapSE` object. However, if you want to save time and you know what you are doing you can set this to `None`, so no connections are reset.

You can pass an existing `DynapseControl` instance to the layer that will handle the interactions with `Cortexcontrol`. If this argument is `None`, a new `DynapseControl` will be instantiated. In this case, you can use the `rpyc_port` argument to define a port at which it should try to establish the connection.

All of these values can be accessed and changed via `<Layer>.value`, where `<Layer>` is the instance of the layer.

### Example
We can set up a simple layer on the chip by only passing input weights and recurrent weights. The weights are chosen so that there is a population of 3 "input neurons" that receive the input and then excite the remaining 6 neurons, which are recurrently connected. This way the constraints mentioned above are satisfied.

In [3]:
import numpy as np

# - Weight matrices: 3 neurons receive external input
#   from 2 channels and stimulate the remaning
#   6 neurons, which are recurrently connected.

weights_in = np.zeros((2, 9))
# Only first 3 neurons receive input
weights_in[:, :3] = np.random.randint(-2, 2, size=(2, 3))

weights_rec = np.zeros((9, 9))
# Excitatory connections from input neurons to rest
weights_rec[:3, 3:] = np.random.randint(3, size=(3, 6))
# Recurrent connecitons between remaining 6 neurons
weights_rec[3:, 3:] = np.random.randint(-2, 2, size=(6, 6))

rlRec = RecDynapSE(weights_in, weights_rec, l_input_core_ids=[0], name="example-layer")

RecDynapSE `example-layer`: Superclass initialized
dynapse_control: RPyC connection established through port 1300.
dynapse_control: RPyC namespace complete.
dynapse_control: RPyC connection has been setup successfully.
DynapseControl: Initializing DynapSE
DynapseControl: Spike generator module ready.
DynapseControl: Poisson generator module ready.
DynapseControl: Time constants of cores [] have been reset.
DynapseControl: Neurons initialized.
	 0 hardware neurons and 1023 virtual neurons available.
DynapseControl: Neuron connector initialized
DynapseControl: Connectivity array initialized
DynapseControl: FPGA spike generator prepared.
DynapseControl ready.
DynapseControl: Not sufficient neurons available. Initializing  chips to make more neurons available.
dynapse_control: Chips 0 have been cleared.
DynapseControl: 1023 hardware neurons available.
Layer `example-layer`: Layer neurons allocated
Layer `example-layer`: Virtual neurons allocated
DynapseControl: Excitatory connections of ty

## Choosing dt
As with all layers, a `RecDynapSE` object's evolution takes place in discrete time steps. This allows to send an `num_timesteps` argument to the `evolve` method, which is consistent with other layer classes and important for the use within a `Network`. Besides, event though the hardware evolves in continuous time, the input events use discrete timesteps. These timesteps are $\frac{10^{-7}}{9} \text{ s} = 11.\bar{1} \cdot 10^{-9} \text{ s}$ and mark the smallest value that can be chosen for the layer timestep `dt`.

Although the effect the timestep size has on computation time is much smaller than with other layers, it is not always advisable to choose the smallest possible value. The reason is that the number of timesteps between two input spikes is currently limited to $2^{16}-1 = 65535$. This means that with dt $= 11.\overline{1} \cdot 10^{-9} \text{ s}$, any section in the  input signal without input spikes longer than about 0.73 milliseconds will cause the layer to throw an exception. Therefore it makes sense to set `dt` to something between $10^{-6}$ and $10^{-4}$ seconds, in order to allow for sufficiently long silent parts in the input while still maintaining a good temporal resolution.

**If these limitations are causing problems contact Felix so that he can implement a way around it.**

## Neurons
The layer neurons of `RecDynapSE` objects directly correspond to physical neurons on the chip. Inputs are sent to the hardware through so-called virtual neurons. Each input channel of the layer corresponds to such a virtual neuron. Every neuron on the DynapSE has a logical ID, which ranges from 0 to 4095 for the physical and from 0 to 1023 for the virtual neurons. 

<!--- (probably not necessary) An input spike to the layer translates to a virtual neuron emitting a spike. If a neuron is set to receive spikes from a neuron with ID $n$, it makes no difference if a virtual or a pysical neuron with this ID is firing. The target neuron will receive the spikes in both cases. Because this can cause unexpected behavior it is recommended that the IDs of the physical neurons are pairwise different from the IDs of of the virtual neurons.
--> 

### Neuron states
Hardware neurons' states change constantly according to the laws of physics, even when the layer is currently not evolving, and there is no state variable that could be read out for all neurons simultaneously. Therefore the `RecDynapSE` has no state vector like other layer classes. The `state` attribute is just a 1D array of `size` zeros.

## Synapses

There are four different synapse types on the DynapSE: fast and slow excitatory as well as fast and slow inhibitory. Each neuron can receive inputs through up to 64 synapses, each of which can be any of the given types. Via `cortexcontrol` the synaptic behavior can be adjusted for each type and for each core, but not for individual synapses.

There is a priori no difference between slow and fast excitatory synapses, so they can be set to have the same behavior. In fact, one could assign shorter time constants to the "slow" excitatory synapses, making them effectively the fast ones. While both excitatory and the fast inhibitory synapses work by adding or subtracting current to the neuron membrane, the slow inhibitory synapses use shunt inhibition and in practice silence a neuron very quickly.

Note that all synapses that are of the same type and that are on the same core have the same weight. Different connection strengths between neurons can only be achieved by setting the same connection multiple times. Therefore the weight matrices `weights_in` and `weights_rec` can only be positive or negative integers and thus determine the number of excitatory and inhibitory connections to a neuron.

In this layer the connections from external input to layer neurons are fast excitatory or inhibitory synapses. Outgoing connections from neurons that receive external input are fast excitatory or inhibitory. Outgoing connecitons from other neurons are slow excitatory and fast inhibitory. For different configurations the `_compile_weights_and_configure` method has to be modified.  Felix can help you with this.

## Simulation

### Evolution in batches
As for now it is not possible to stream events (input spikes) continuously to the DynapSE. Therefore a group of events is transferred to the hardware, temporarily stored there and then translated to spikes of virtual neurons, with temporal order and inter-spike intervals matching the input signal.

The number of events that can be sent at once is limited. To allow for arbitrarily long layer evolution times, the input can be split into batches, during each of which a number of events is sent and "played back" to the hardware.

Note that because the hardware neurons keep evolving after a batch ends, their state (membrane potentials etc.) will have changed until the next batch starts. These discontinuities could be problematic for some simulations. In particular if the input data consists of trials, no trial should be divided over two batches.

The second way of splitting batches considers this scenario and by splitting the input only at the beginnings of trials. The number of trials in a batch is determined by the layer's `nMaxTrialPerBatch` attribute. If a batch with this number of trials contains more events or lasts longer than allowed, the number of trials is reduced accordingly. This method is used if the input time series has a `trial_start_times` attribute, that indicates the start time of each trial, and if `max_num_trials_batch` is not `None`.

## Resetting

## Internal methods

```
_batch_input_data(
    self, ts_input: TSEvent, num_timesteps: int, verbose: bool = False
) -> (np.ndarray, int)
```
This mehtod is called by evolve, which passes it the evolution input `ts_input` and the number of evolution timesteps `num_timesteps`. It splits the input into batches according to the maximum duration, number of events and number of trials and returns a generator that for each batch yields the timesteps and channels of the input events in the batch, the time step at which the batch begins and teh duration of the batch.

```
_compile_weights_and_configure(self)
```
Configures the synaptic connections on the hardware according to the layer weights.

## Class member overview

### Methods

| arguments: | Description |
|--------|-------------|
| `_batch_input_data` | Split evolution into batches and return generator |
| `_compile_weights_and_configure` | Configure hardware synaptic connections |
| `evolve` | Evolve layer |
| `reset_all` | Reset layer time to 0|
| `reset_state` | Do nothing. |
| `reset_time` | Reset layer time to 0 |

*Internal methods of parent class* `Layer` *are listed in corresponding documentation.*

### Attributes

Each argument that described in section Instantiation has a corresponding attribute that can be accessed by `<Layer>.<attribute>`, where `<Layer>` is the layer instance and `<attribute>` the argument name. Furthermore there are a few internal attributes:

| Attribute name | Description |
|----------------|-------------|
| `_vHWNeurons` | 1D-Array of hardware neurons used for the layer. |
| `vVirtualNeurons` | 1D-Array of virtual neurons used for the layer. |